In [ ]:
# installing the accelerate library
!pip install accelerate

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 290.1/290.1 kB 2.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.7/23.7 MB 17.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 823.6/823.6 kB 28.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.1/14.1 MB 13.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 731.7/731.7 MB 1.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 410.6/410.6 MB 1.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 121.6/121.6 MB 2.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.5/56.5 MB 2.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 124.2/124.2 MB 1.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 196.0/196.0 MB 1.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 166.0/166.0 MB 2.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 99.1/99.1 kB 3.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━

In [ ]:
!pip install datasets

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 510.5/510.5 kB 3.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 17.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 11.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 16.3 MB/s eta 0:00:00


In [ ]:
import re

def mark_branch_hints(code):
    # Combined regular expression pattern for 'likely' and 'unlikely' hints

    if_statement = code.split('\n')[-3:]
    if_statement = '\n'.join(if_statement)
    pattern = re.compile(r'\[\[(likely|unlikely)\]\]')
    labels = {'likely': 0, 'unlikely':1, None: 2}
    # List to hold labels
    # Function to replace the patterns
    labels_list = []

    # Function to replace the patterns
    def remove_tags(match):
        hint_type = match.group(1)  # Get the hint type ('likely' or 'unlikely')
        labels_list.append(labels[hint_type])  # Add the corresponding label to the list
        return ""  # Remove

    # Replace 'likely' and 'unlikely' with placeholders and populate labels_list
    pattern.sub(remove_tags, if_statement)
    processed_code = pattern.sub("", code)

    # Check if no hints were found and assign None label
    if not labels_list:
        labels_list.append(labels[None])

    return {"text": processed_code, "label": labels_list[-1]}
    # if labels:
    #   return {"code": code, "labels": labels[label]}
    # else:
    #   return {"code

# Example C++ code
cpp_code = """
if (condition) [[unlikely]]{
    // Likely branch
} else {
    // Unlikely branch
}
"""

# Process the code to mark the hints
marked_code = mark_branch_hints(cpp_code)
print(marked_code)


{'text': '\nif (condition) {\n    // Likely branch\n} else {\n    // Unlikely branch\n}\n', 'label': 2}


In [ ]:
from datasets import load_dataset
raw_dataset = load_dataset("parquet", data_files="cxx-likely-unlikely.parquet")


Generating train split: 0 examples [00:00, ? examples/s]

In [ ]:
extracted_dataset = []
for i in range(len(raw_dataset["train"])):
  extracted_dataset.append(raw_dataset["train"][i]["code"])
extracted_dataset = extracted_dataset

In [ ]:
final_dataset = []

likely_count = 0
unlikely_count = 0
none_count = 0
bruh_count = 0

for i in range(len(extracted_dataset)):
    result = mark_branch_hints(extracted_dataset[i])
    label = result["label"]
    if label == 0:
        likely_count += 1
    elif label == 1:
        unlikely_count += 1
    elif label == 2:
        none_count += 1
    final_dataset.append(result)
print(final_dataset[100].keys())
print("Likely: " + str(likely_count))
print("Unlikely: " + str(unlikely_count))
print("None: " + str(none_count))


dict_keys(['text', 'label'])
Likely: 8297
Unlikely: 23489
None: 28380


In [ ]:
from transformers import AutoModelForSequenceClassification, AutoTokenizer, Trainer, TrainingArguments
from datasets import Dataset
import pandas as pd

from sklearn.metrics import accuracy_score, precision_recall_fscore_support

model_name = "microsoft/codebert-base"

labels = {'likely': 0, 'unlikely':1, None: 2}
# data = final_dataset[:10]
data = final_dataset
print(data)
df = pd.DataFrame(data)


train_df = df.sample(frac=0.7)
temp_df = df.drop(train_df.index)
valid_df = temp_df.sample(frac=0.66)
test_df = temp_df.drop(valid_df.index)


tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=3)
def tokenize_function(examples):
    return tokenizer(examples["text"], padding="max_length", truncation=True, max_length=512)

def prepare_dataset(df):
    ds = Dataset.from_pandas(df)
    ds = ds.map(tokenize_function, batched=True)
    ds.set_format(type='torch', columns=['input_ids', 'attention_mask', 'label'])
    return ds

def compute_metrics(pred):
    labels = pred.label_ids
    preds = pred.predictions.argmax(-1)
    precision, recall, f1, _ = precision_recall_fscore_support(labels, preds, average='weighted')
    acc = accuracy_score(labels, preds)
    return {
        'accuracy': acc,
        'f1': f1,
        'precision': precision,
        'recall': recall
    }


train_dataset = prepare_dataset(train_df)
valid_dataset = prepare_dataset(valid_df)
test_dataset = prepare_dataset(test_df)

training_args = TrainingArguments(
    output_dir="./results",
    num_train_epochs=10,
    per_device_train_batch_size=14,
    per_device_eval_batch_size=14,
    warmup_steps=500,
    weight_decay=0.01,
    logging_dir="./logs",
    logging_steps=10,
    evaluation_strategy="epoch",
    save_strategy="epoch",
    load_best_model_at_end=True,
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=valid_dataset,
    compute_metrics=compute_metrics,
)

trainer.train()

results = trainer.evaluate(test_dataset)
print(f'RESULTS: \n{results}')

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/25.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/498 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

...

Map:   0%|          | 0/11913 [00:00<?, ? examples/s]

Map:   0%|          | 0/6137 [00:00<?, ? examples/s]

/usr/local/lib/python3.10/dist-packages/accelerate/accelerator.py:432: FutureWarning: Passing the following arguments to `Accelerator` is deprecated and will be removed in version 1.0 of Accelerate: dict_keys(['dispatch_batches', 'split_batches', 'even_batches', 'use_seedable_sampler']). Please pass an `accelerate.DataLoaderConfiguration` instead: 
dataloader_config = DataLoaderConfiguration(dispatch_batches=None, split_batches=False, even_batches=True, use_seedable_sampler=True)
  warnings.warn(


Epoch,Training Loss,Validation Loss


Epoch,Training Loss,Validation Loss


In [ ]:
# prompt: mount to drive

from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive
